# Analyze and plot correlations among fitness estimates and DMS measurements

Import Python modules:

In [ ]:
import itertools
import os

import altair as alt

import numpy

import pandas as pd

import yaml

_ = alt.data_transformers.disable_max_rows()

Now get variables from `snakemake`:

In [ ]:
if "snakemake" not in globals() and "snakemake" not in locals():
    # variables set manually for interactive debugging
    aa_fitness_csv = "../results/aa_fitness/aa_fitness.csv"
    neher_fitness_csv = "../data/Neher_aa_fitness.csv"
    plotsdir = "../results/fitness_dms_corr/plots"
    
    with open("../config.yaml") as f:
        config = yaml.safe_load(f)
    min_expected_count = config["min_expected_count"]
    dms_datasets = config["dms_datasets"]
    
    dms_datasets_csvs = {
        dms_dataset: f"../results/dms/{dms_dataset}/processed.csv"
        for dms_dataset in dms_datasets
    }
    
else:
    # get variables from `snakemake` when running pipeline
    aa_fitness_csv = snakemake.input.aafitness
    neher_fitness_csv = snakemake.input.neher_fitness
    plotsdir = snakemake.output.plotsdir
    min_expected_count = snakemake.params.min_expected_count
    dms_datasets = snakemake.params.dms_datasets
    dms_datasets_csvs = {
        dms_dataset: getattr(snakemake.input, dms_dataset)
        for dms_dataset in dms_datasets
    }
    
os.makedirs(plotsdir, exist_ok=True)

Read the datasets:

In [ ]:
aa_fitness = pd.read_csv(aa_fitness_csv)
neher_fitness = pd.read_csv(neher_fitness_csv)
dms_dataset_dfs = {
    dms_dataset: pd.read_csv(f) for dms_dataset, f in dms_datasets_csvs.items()
}

## Correlation of fitness estimates with Neher fitness estimates
First examining correlations between amino-acid fitness values from the current approach and the Neher estimates (which are just for spike).
To do this, we have to get the estimates to share a common "wildtype" identity, which we choose as whatever is the Neher et al identity.
Note this is just for spike as we only have Neher et al estimates for spike:

In [ ]:
spike_fitness = (
    aa_fitness
    .query("gene == 'S'")
    .rename(columns={"aa_site": "site"})
    [["site", "aa", "fitness", "expected_count"]]
)

neher_corr_df = (
    neher_fitness
    .query("aa_fitness.notnull()")
    .assign(
        wildtype=lambda x: x["mutation"].str[0],
        site=lambda x: x["mutation"].str[1: -1].astype(int),
        mutant=lambda x: x["mutation"].str[-1],
    )
    .rename(columns={"aa_fitness": "Neher fitness effect"})
    [["wildtype", "site", "mutant", "Neher fitness effect"]]
    .merge(
        spike_fitness.rename(
            columns={
                "aa": "wildtype",
                "fitness": "wt_fitness",
                "expected_count": "expected_count_wt",
            }
        ),
        on=["site", "wildtype"],
        how="inner",
        validate="many_to_one",
    )
    .merge(
        spike_fitness.rename(
            columns={
                "aa": "mutant",
                "fitness": "mut_fitness",
                "expected_count": "expected_count_mut",
            }
        ),
        on=["site", "mutant"],
        how="inner",
        validate="many_to_one",
    )
    .query("wildtype != mutant")
    .assign(
        expected_count=lambda x: numpy.minimum(
            x["expected_count_wt"], x["expected_count_mut"],
        ),
        fitness_effect=lambda x: x["mut_fitness"] - x["wt_fitness"],
        mutation=lambda x: x["wildtype"] + x["site"].astype(str) + x["mutant"],
    )
    [["mutation", "fitness_effect", "Neher fitness effect", "expected_count"]]
)

display(neher_corr_df.head())

neher_corr_df.corr()

Plot correlation.
First define a function to do this, then make the plots:

In [ ]:
def plot_corr_scatters(corr_df_tidy, ncols=4, ignore_pairs=set()):
    "Plot set of correlation scatters."""
    
    subsets = corr_df_tidy["subset"].unique()
    
    corr_df_wide = (
        corr_df_tidy
        .pivot_table(
            index=["mutation", "expected_count"],
            values="delta_fitness",
            columns="subset",
        )
        .reset_index()
        .assign(
            measured_in_all=lambda x: x[subsets].notnull().all(axis=1).map(
                {True: 1, False: 0}
            )
        )
        .rename(columns={subset: f"delta_fitness {subset}" for subset in subsets})
    )

    measured_in_all = alt.selection_single(
        fields=["value"],
        bind=alt.binding_radio(
            options=[0, 1],
            labels=["no", "yes"],
            name="only show mutations measured in all subsets?",
        ),
        init={"value": 0},
    )

    expected_count_selection = alt.selection_single(
        bind=alt.binding_range(
            min=1,
            max=min(5 * min_expected_count, corr_df_tidy["expected_count"].quantile(0.9)),
            step=1,
            name="minimum expected count",
        ),
        fields=["cutoff"],
        init={"cutoff": min_expected_count},
    )
    
    highlight = alt.selection_single(
        on="mouseover",
        fields=["mutation"],
        empty="none",
    )
    
    corr_charts = []
    base_chart = alt.Chart(corr_df_wide)
    
    if len(subsets) > 2:
        base_chart = base_chart.transform_filter(
            measured_in_all["value"] <= alt.datum["measured_in_all"]
        )
    
    for subset2, subset1 in itertools.combinations(subsets, 2):
        if {subset1, subset2} in ignore_pairs:
            continue
        
        base = (
            base_chart
            .encode(
                x=alt.X(
                    f"delta_fitness {subset1}",
                    title=subset1,
                    scale=alt.Scale(zero=False, nice=False),
                ),
                y=alt.Y(
                    f"delta_fitness {subset2}",
                    title=subset2,
                    scale=alt.Scale(zero=False, nice=False),
                ),
                tooltip=[
                    "mutation",
                    alt.Tooltip(
                        f"delta_fitness {subset1}", title=subset1, format=".4g",
                    ),
                    alt.Tooltip(
                        f"delta_fitness {subset2}", title=subset2, format=".4g",
                    ),
                    alt.Tooltip("expected_count"),
                    alt.Tooltip("measured_in_all"),
                ],
            )
            .mark_circle(opacity=0.3)
            .properties(width=200, height=200)
            .transform_filter(
                alt.datum["expected_count"] >= expected_count_selection["cutoff"] - 1e-6
            )
        )
    
        scatter = (
            base
            .encode(
                size=alt.condition(highlight, alt.value(85), alt.value(30)),
                opacity=alt.condition(highlight, alt.value(1), alt.value(0.3)),
                strokeWidth=alt.condition(highlight, alt.value(1.5), alt.value(0)),
            )
            .mark_circle(stroke="black")
        )
    
        # regression line and correlation coefficient: https://stackoverflow.com/a/60239699
        line = (
            base
            .transform_regression(
                f"delta_fitness {subset1}",
                f"delta_fitness {subset2}",
            )
            .mark_line(color="orange", clip=True)
        )
    
        # show correlation coefficient
        params_r = (
            base
            .transform_regression(
                f"delta_fitness {subset1}",
                f"delta_fitness {subset2}",
                params=True,
            )
            .transform_calculate(
                r=alt.expr.sqrt(alt.datum["rSquared"]),
                label='"r = " + format(datum.r, ".3f")',
            )
            .mark_text(align="left", color="orange", fontWeight="bold")
            .encode(
                x=alt.value(5),
                y=alt.value(8),
                text=alt.Text("label:N"),
            )
        )
        
        # show number of points
        params_n = (
            base
            .transform_filter(
                (~alt.expr.isNaN(alt.datum[f"delta_fitness {subset1}"]))
                & (~alt.expr.isNaN(alt.datum[f"delta_fitness {subset2}"]))
            )
            .transform_calculate(dummy=alt.datum[f"delta_fitness {subset1}"])
            .transform_aggregate(n="valid(dummy)")
            .transform_calculate(label='"n = " + datum.n')
            .mark_text(align="left", color="orange", fontWeight="bold")
            .encode(
                x=alt.value(5),
                y=alt.value(20),
                text=alt.Text("label:N"),
            )
        )
    
        chart = (
            (scatter + line + params_r + params_n)
            .add_selection(expected_count_selection)
            .add_selection(highlight)
        )
    
        corr_charts.append(chart)

    rows = []
    for i in range(0, len(corr_charts), ncols):
        rows.append(alt.hconcat(*corr_charts[i: i + ncols]))
    corr_chart = alt.vconcat(*rows).configure_axis(grid=False)
    
    if len(subsets) > 2:
        corr_chart = corr_chart.add_selection(measured_in_all)

    return corr_chart


neher_corr_df_tidy = (
    neher_corr_df
    .rename(columns={"fitness_effect": "fitness effect"})
    .melt(
        id_vars=["mutation", "expected_count"],
        var_name="subset",
        value_name="delta_fitness",
    )
)

neher_corr_chart = plot_corr_scatters(neher_corr_df_tidy)

neher_corr_chart.save(os.path.join(plotsdir, "neher_fitness_corr.html"))

neher_corr_chart

## Correlation of current and Neher fitness estimates with each spike deep mutational scanning dataset
Do the current or Neher fitness estimates correlate better with deep mutational scanning?
For each deep mutational scanning dataset, correlate the fitness effects estimated from the current study and the Neher et al fitness estimates:

In [ ]:
for dms_dataset, dms_df in dms_dataset_dfs.items():
    
    dms_config = dms_datasets[dms_dataset]
    gene = dms_config["gene"]
    
    if gene != "S":
        continue  # we only have Neher estimates for spike
    
    desc = dms_config["description"]
    print(f"\nAnalyzing {dms_dataset=} for {gene=}")
    if "filter_cols" in dms_config:
        for filter_col, filter_val in dms_config["filter_cols"].items():
            print(f"Filtering for {filter_col} >= {filter_val}")
            dms_df = dms_df.query(f"{filter_col} >= {filter_val}").drop(columns=filter_col)
            
    dms_cols = {
        c: f"DMS {c.replace('_', ' ')} {desc}"
        for c in dms_df
        if c not in {"site", "wildtype", "mutant"}
    }
    
    # do not correlate total DMS effect with individual assays
    ignore_pairs = {frozenset({f"DMS effect {desc}", c}) for c in dms_cols.values()}
    
    dms_df = dms_df.rename(columns=dms_cols)
    if dms_df["site"].dtype != int:
        # only keep integer sites as they are only ones with fitness estimates
        dms_df = dms_df[dms_df["site"].str.fullmatch("\d+")]
        dms_df["site"] = dms_df["site"].astype(int)
    
    gene_fitness = (
        aa_fitness
        .query("gene == @gene")
        .rename(columns={"aa_site": "site"})
        [["site", "aa", "fitness", "expected_count"]]
    )
    
    corr_df_wide = (
        dms_df
        .merge(
            gene_fitness.rename(
                columns={
                    "aa": "wildtype",
                    "fitness": "wt_fitness",
                    "expected_count": "expected_count_wt",
                }
            ),
            on=["site", "wildtype"],
            how="inner",
            validate="many_to_one",
        )
        .merge(
            gene_fitness.rename(
                columns={
                    "aa": "mutant",
                    "fitness": "mut_fitness",
                    "expected_count": "expected_count_mut",
                }
            ),
            on=["site", "mutant"],
            how="inner",
            validate="many_to_one",
        )
        .query("wildtype != mutant")
        .assign(
            expected_count=lambda x: numpy.minimum(
                x["expected_count_wt"], x["expected_count_mut"],
            ),
            fitness_effect=lambda x: x["mut_fitness"] - x["wt_fitness"],
            mutation=lambda x: x["wildtype"] + x["site"].astype(str) + x["mutant"],
        )
        [["mutation", "fitness_effect", "expected_count", *dms_cols.values()]]
        .rename(columns={"fitness_effect": "fitness effect"})
    )
    
    if gene == "S":
        corr_df_wide = corr_df_wide.merge(
            neher_fitness
            [["mutation", "aa_fitness"]]
            .rename(columns={"aa_fitness": "Neher fitness effect"})
        )
    
    corr_df = (
        corr_df_wide
        .melt(
            id_vars=["mutation", "expected_count"],
            var_name="subset",
            value_name="delta_fitness",
        )
    )

    corr_chart = plot_corr_scatters(corr_df, ignore_pairs=ignore_pairs)
    
    corr_chart.save(os.path.join(plotsdir, f"dms_neher_fitness_{dms_dataset}_corr.html"))
    
    display(corr_chart)

## Per gene correlations with DMS
Plot the per-gene correlations both within every measurement of the DMS and just the overall effects from each study:

In [ ]:
dms_datasets_by_gene = {
    gene: [
        dms_dataset
        for dms_dataset, dms_config in dms_datasets.items()
        if dms_config["gene"] == gene
    ]
    for gene in aa_fitness["gene"].unique()
}

for gene, gene_dms_datasets in dms_datasets_by_gene.items():
    if not gene_dms_datasets:
        continue
        
    print(f"\nPlotting correlations for {gene=}")
    
    gene_fitness = (
        aa_fitness
        .query("gene == @gene")
        .rename(columns={"aa_site": "site"})
        [["site", "aa", "fitness", "expected_count"]]
    )
    
    ignore_pairs = set()
    
    corr_dfs = []
    
    for dms_dataset in gene_dms_datasets:
        dms_df = dms_dataset_dfs[dms_dataset]
        dms_config = dms_datasets[dms_dataset]
        desc = dms_config["description"]
        print(f"Analyzing {dms_dataset=} for {gene=}")
        if "filter_cols" in dms_config:
            for filter_col, filter_val in dms_config["filter_cols"].items():
                print(f"Filtering for {filter_col} >= {filter_val}")
                dms_df = dms_df.query(f"{filter_col} >= {filter_val}").drop(columns=filter_col)
            
        dms_cols = {
            c: f"DMS {c.replace('_', ' ')} {desc}"
            for c in dms_df
            if c not in {"site", "wildtype", "mutant"}
        }
    
        # do not correlate total DMS effect with individual assays
        ignore_pairs = ignore_pairs.union(
            {frozenset({f"DMS effect {desc}", c}) for c in dms_cols.values()}
        )
    
        dms_df = dms_df.rename(columns=dms_cols)
        if dms_df["site"].dtype != int:
            # only keep integer sites as they are only ones with fitness estimates
            dms_df = dms_df[dms_df["site"].str.fullmatch("\d+")]
            dms_df["site"] = dms_df["site"].astype(int)
    
        corr_df_wide = (
            dms_df
            .merge(
                gene_fitness.rename(
                    columns={
                            "aa": "wildtype",
                        "fitness": "wt_fitness",
                        "expected_count": "expected_count_wt",
                    }
                ),
                on=["site", "wildtype"],
                how="inner",
                validate="many_to_one",
            )
            .merge(
                gene_fitness.rename(
                    columns={
                        "aa": "mutant",
                        "fitness": "mut_fitness",
                        "expected_count": "expected_count_mut",
                    }
                ),
                on=["site", "mutant"],
                how="inner",
                validate="many_to_one",
            )
            .query("wildtype != mutant")
            .assign(
                expected_count=lambda x: numpy.minimum(
                    x["expected_count_wt"], x["expected_count_mut"],
                ),
                fitness_effect=lambda x: x["mut_fitness"] - x["wt_fitness"],
                mutation=lambda x: x["wildtype"] + x["site"].astype(str) + x["mutant"],
            )
            [["mutation", "fitness_effect", "expected_count", *dms_cols.values()]]
            .rename(columns={"fitness_effect": "fitness effect"})
        )
    
        corr_dfs.append(
            corr_df_wide
            .melt(
                id_vars=["mutation", "expected_count"],
                var_name="subset",
                value_name="delta_fitness",
            )
        )
        
    corr_df = pd.concat(corr_dfs).drop_duplicates().reset_index(drop=True)

    print("\nPlot of all DMS measurements:")
    corr_chart_all = plot_corr_scatters(corr_df, ignore_pairs=ignore_pairs)
    
    corr_chart_all.save(os.path.join(plotsdir, f"dms_{gene.split()[0]}_all_corr.html"))
    
    display(corr_chart_all)
    
    print("\nPlot of just overall effects for each DMS study:")
    corr_chart_overall = plot_corr_scatters(
        corr_df.query(
            "subset.str.startswith('DMS effect') or subset == 'fitness effect'"
        ),
    )
    
    corr_chart_overall.save(os.path.join(plotsdir, f"dms_{gene.split()[0]}_corr.html"))
    
    display(corr_chart_overall)